## Intro to TableGen Part 2: mutliclass and TODO: builtin functions ?

This turorial builds on part 1 where you learned about classes, defs, types and let.

## MultiClasses

A powerful tool for generating classes is `multiclass`. Using it saves us a lot of repetition.

I'm going to use a registers example again and here's all the background you'll need. Arm's general registers have "X" and "W" names for the 64 and 32 bit views on a register respectively.

If I were to encode this without a multiclass I'd have to repeat myself a lot. Delcaring each "X" and "W" register separately.

In [12]:
%reset
class Register {
    int size=4;
}
let size=8 in {
    def X0: Register {}
    def X1: Register {}
    // Repeat until X31...
}
def W0: Register {}
def W1: Register {}
// Repeat until W31...

------------- Classes -----------------
class Register {
  int size = 4;
}
------------- Defs -----------------
def W0 {	// Register
  int size = 4;
}
def W1 {	// Register
  int size = 4;
}
def X0 {	// Register
  int size = 8;
}
def X1 {	// Register
  int size = 8;
}


`multiclass` saves you the repetition. `multiclass` produces mutliple defs when you use it, so we can use it to make an "X" and a "W" register for us.

To make defs from a `multiclass` we use `defm`. Simply `def` but with `m` for "multiple". In this case we provide one index `0` then we get an `X0` and a `W0` back.

In [13]:
%reset
class Register <int _size, string _name> {
    int size = _size;
    string name = _name;
}

multiclass RegisterSizes<string name> {
    def _8Byte : Register<8, !strconcat("X", name)>;
    def _4Byte : Register<4, !strconcat("W", name)>;
}

defm Register_0: RegisterSizes<"0">;
// Repeat until 31...

------------- Classes -----------------
class Register<int Register:_size = ?, string Register:_name = ?> {
  int size = Register:_size;
  string name = Register:_name;
}
------------- Defs -----------------
def Register_0_4Byte {	// Register
  int size = 4;
  string name = "W0";
}
def Register_0_8Byte {	// Register
  int size = 8;
  string name = "X0";
}


The name of the new defs is the concatenation of the base class (`Register`) and the def name within the multiclass. So we got `Register_0_8Byte` and `Register_0_4Byte`.

Using the multiclass has cut the number of `defs` in half (and we could further trim this down using the functions shown later).

## Mutliclass In Other Languages

It may be helpful to relate `multiclass` to concepts from other languages. As the name is somewhat misleading.

`mutliclass` is not a class in itself. It is more like a function that returns multiple instances of a class. Thinking about it that way, there are plenty of parallels to other languages.

For instance in C++ you might use a factory function:
```
#include <string>
#include <utility>

struct Register {
    Register(int size, std::string name):
        m_size(size), m_name(name) {}

    int m_size;
    std::string m_name;
};

std::pair<Register, Register> make_reg_defs(std::string index) {
    return {
        Register(8, "X" + index),
        Register(4, "W" + index),
    };
}
```

Very similar in Python:
```
class Register(object):
    def __init__(self, size, name):
        self.size = size
        self.name = name
        
def make_reg_defs(index):
    return [Register(8, "X" + index), Register(4, "W" + index)]
    
print(make_reg_defs("0")[0].name)
# prints "X0"
```

The overall theme is that you make one call and get multiple things back.

* `def` - define one thing
* `defm` - define many things

## defset

`defset` takes the defs within `{...}` and defines them as usual but also adds them to a global variable.

`defset <type of the set> <name of the set> = { <do some def/defm here> }`

In [57]:
%reset

class Counter <int c> {
    int count = c;
}

defset list<Counter> Counters = {
    def c0 : Counter<0>;
    def c1 : Counter<1>;
    def c2 : Counter<2>;
}

------------- Classes -----------------
class Counter<int Counter:c = ?> {
  int count = Counter:c;
}
------------- Defs -----------------
def c0 {	// Counter
  int count = 0;
}
def c1 {	// Counter
  int count = 1;
}
def c2 {	// Counter
  int count = 2;
}


As this new variable `Counters` is not itself a class or record, you won't see it in the default `llvm-tblgen` output. Instead this feature helps you create other records by processing the set of defs.

Let's use it to make one final def that is the total of all the counters. What we want to do is iterate over all the defs in `Counters` and add up their counts as we go.

In [58]:
def final_c {
    int total = !foldl(0, Counters, accumulator, counter, !add(accumulator, counter.count));
}

------------- Classes -----------------
class Counter<int Counter:c = ?> {
  int count = Counter:c;
}
------------- Defs -----------------
def c0 {	// Counter
  int count = 0;
}
def c1 {	// Counter
  int count = 1;
}
def c2 {	// Counter
  int count = 2;
}
def final_c {
  int total = 3;
}


I've used a few functions we haven't talked about. `foldl` will be familiar to people with functional programming experience and `!add` simply adds two things together. We will talk more about these and other functions later.

For now take it for granted that total is calculated by starting with `0` and adding the `count` of each `Counter` from `Counters` to it.

The point is that without `defset` to group them under one name, we would have had to do it in a much less flexible manner.

In [59]:
def final_c_no_defset {
    int total = !add(c0.count, c1.count, c2.count);
}

------------- Classes -----------------
class Counter<int Counter:c = ?> {
  int count = Counter:c;
}
------------- Defs -----------------
def c0 {	// Counter
  int count = 0;
}
def c1 {	// Counter
  int count = 1;
}
def c2 {	// Counter
  int count = 2;
}
def final_c {
  int total = 3;
}
def final_c_no_defset {
  int total = 3;
}
